# Preprocessing for Process Prediction Pipeline

In [1]:
#%load_ext autoreload
#%autoreload 2
# %load_ext line_profiler
#%matplotlib inline

In [2]:
#export
from exp.eventlog import *
from exp.dl_utils import *
from collections import OrderedDict
from collections import defaultdict
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import shutil

In [3]:
#export
def store_df(df, path):
    # Prepend dtypes to the top of df
    df.to_pickle(path+'.pickle')

def load_df(path):
    return pd.read_pickle(path+'.pickle')

In [4]:
#export
def store_df(df, path):
    # Prepend dtypes to the top of df
    df2 = df.copy()
    df2.loc[-1] = df2.dtypes
    df2.index = df2.index + 1
    df2.sort_index(inplace=True)
    # Then save it to a csv
    df2.to_csv(path, index=False)

def load_df(path):
    # Read types first line of csv
    dtypes = {key:value for (key,value) in pd.read_csv(path,    
              nrows=1).iloc[0].to_dict().items() if 'date' not in value}

    parse_dates = [key for (key,value) in pd.read_csv(path, 
                   nrows=1).iloc[0].to_dict().items() if 'date' in value]
    # Read the rest of the lines with the types from above
    return pd.read_csv(path, dtype=dtypes, parse_dates=parse_dates, skiprows=[1])

In [5]:
#export
def mse(output, targ): 
    if not torch.is_tensor(output): output=tensor(output)
    if not torch.is_tensor(targ): targ=tensor(targ)
    return (output.squeeze(-1) - targ).pow(2).mean()

In [6]:
#export
_unknown=0
_padding=1
_special_token=['__UNKNOWN__','__PADDING__']

class LabelEncoder():
    def __init__(self,special_token=_special_token,unknown=_unknown,padding=_padding): 
        self.unknown=unknown
        self.special_token=special_token
    def fit(self, x):
        self.vocab=list(np.unique(x))
        for o in reversed(self.special_token):
            if o in self.vocab: self.vocab.remove(o)
            self.vocab.insert(0, o)
        if getattr(self, 'otoi', None) is None:
            self.otoi = defaultdict(int,{v:k for k,v in enumerate(self.vocab)})
    def transform(self,x): return [self.otoi.get(o, self.unknown) for o in x]
    def inverse_transform(self,x): return [self.vocab[i] for i in x]

In [7]:
#train=['a', 'b','c']
#test=['a','c','d']
#le=LabelEncoder()
#le.fit(train)
#print(le.vocab)
#print(le.otoi)
#trans=le.transform(test)
#print(trans)
#print(le.inverse_transform(trans))

In [8]:
#export
def random_split_traces(d,split=0.8,trace_id='trace_id'):
    traces=d[trace_id].drop_duplicates()
    shuffled=traces.iloc[np.random.permutation(len(traces))].values
    split=int(len(traces)*split)
    return shuffled[:split],shuffled[split:]

In [9]:
#export
def label_encode(x,label_encoders):
    x[pd.isnull(x)]  = 'is missing' 
    if not x.name in label_encoders:
        label_encoders[x.name]=LabelEncoder()
        label_encoders[x.name].fit(x)
    return label_encoders[x.name].transform(x)

In [10]:
#export
def calc_suffix(groups,col):
    suffix=[]
    for n,g in groups:
        c=g[col].values
        for i in range(len(c)):
            suffix.append(list(c[i:][1:]))
    return suffix 

In [11]:
#tmp=pd.DataFrame({'c':[1,2,3,4,5,6]})
#tmp.index=['a','b','a','b','a','b']
#tmp['suffix_correct']=[[3,5],[4,6],[5],[6],[],[]]



#tmp.sort_index(inplace=True)
#g=tmp.groupby(tmp.index,sort=False)

#tmp['suffix_calc']=calc_suffix(g,'c')
#tmp['next activity']=g['c'].shift(-1,fill_value=-1).astype(int) 
#tmp['outcome']=g['c'].transform('last')
#tmp

In [12]:
#export
def get_trace_df(t,df,label_encoders,trace_id='trace_id'): 
    res=df[df[trace_id].isin(t)].copy()
    g=res.groupby(trace_id,sort=False)
    res[['activity','resource']]=res[['activity:name','org:resource']].apply(lambda x: label_encode(x,label_encoders))
    res['relative timestamp']=g['time:timestamp'].diff().shift(-1) / np.timedelta64(1, 's')
    res["relative timestamp"].fillna(0, inplace=True) # fills every NaN at end of trace with 0
    # the value at [-2] is the same as at [-3] due to the 0 of relative timestamp
    res["next relative timestamp"] = g["relative timestamp"].transform(lambda x: x.cumsum().shift(-1))
    res["next relative timestamp"].fillna(0, inplace=True)
    res['duration to end']=g['time:timestamp'].transform(lambda x: (x.iloc[-1]-x) / np.timedelta64(1, 's'))
    res['next activity']=g['activity'].shift(-1,fill_value=-1).astype(int) 
    res['next resource']=g['resource'].shift(-1,fill_value=-1).astype(int) 
    res['last resource']=g['resource'].transform('last')
    res['outcome']=g['activity'].transform('last')
    res['activity suffix']=calc_suffix(g,'activity')
    res['resource suffix']=calc_suffix(g,'resource')

    return res

In [13]:
#export
def preprocess_and_train_test_split(dataset,cache=False,override=False): 
    dataset_name=dataset.split('=')[-1].split('.')[0]
    path=f"./.cache/datasets/{dataset_name}"
    if cache and os.path.isdir(path) and not override:
        train_df=load_df(f"{path}/train")
        test_df=load_df(f"{path}/test")
        return train_df, test_df
    log=import_xes(untar_data(dataset))
    df=pd.merge(log.events,log.traceAttributes,left_on='trace_id',right_index=True)
    df['activity:name']=df['concept:name']+'_'+df['lifecycle:transition']
    # missing values (NaN) in duration to next event einfüllen:
    #df['duration to next event'].fillna(0, inplace=True) 
    label_encoders={}
    train_trace_ids,test_trace_ids=random_split_traces(df,0.9)
    train_df=get_trace_df(train_trace_ids,df,label_encoders)
    test_df=get_trace_df(test_trace_ids,df,label_encoders)
    if cache and (not os.path.isdir(path) or override):
        if override:  shutil.rmtree(path)
        try:
            os.makedirs(path)
        except OSError:
            print ("Creation of the directory %s failed" % path)
        else:
            print ("Successfully created the directory %s" % path)
            store_df(train_df,f"{path}/train")
            store_df(test_df,f"{path}/test")
    return train_df,test_df

In [14]:
#%%time
#train_df,test_df=preprocess_and_train_test_split(URLs.BPIC_2012,cache=True,override=True)

In [19]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 000_PPP_Preprocessing.ipynb

Converted 000_PPP_Preprocessing.ipynb to exp/Preprocessing.py
